In [223]:
# imports
import re
import random
import time
import tsplib95
import numpy as np
from typing import Tuple, List, Dict

# for ploting purposes
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [224]:
# env vars
GR_READ = True
DATA_SET_NAME = '../data/gr137.tsp'


In [225]:
# read data
graph = []
coords = []
nodes = []
if not GR_READ:
    with open(DATA_SET_NAME,'r') as file:
        lines = file.readlines()
        graph = [
            list(
                map(
                    lambda i: int(i),
                    re.sub(' +',' ',line.replace('\n','').rstrip().lstrip()).split(' ')
                )
            ) for line in lines
        ]
else :
    instance = tsplib95.load(DATA_SET_NAME)
    nodes = list(instance.get_nodes())
    coords = np.array([instance.node_coords[n] for n in nodes])
    dist_matrix= np.array(
            [[instance.get_weight(n1, n2) for n2 in nodes] for n1 in nodes]
        ).tolist()
    graph = dist_matrix
graph

[[1,
  808,
  1161,
  1766,
  2993,
  2869,
  3092,
  3473,
  3240,
  3788,
  3071,
  5034,
  4989,
  5053,
  5005,
  5482,
  5536,
  3186,
  3326,
  4153,
  4213,
  4722,
  4894,
  4204,
  4962,
  4533,
  4959,
  5313,
  4270,
  4409,
  4685,
  4948,
  5222,
  5525,
  5887,
  4792,
  4917,
  5145,
  5517,
  6048,
  5036,
  5336,
  5309,
  5860,
  5458,
  5519,
  5558,
  5597,
  6309,
  6835,
  7008,
  5654,
  6063,
  6151,
  6596,
  6501,
  6583,
  6851,
  6919,
  6939,
  7097,
  7179,
  7207,
  7005,
  7419,
  7642,
  7785,
  7811,
  8049,
  8359,
  8621,
  7033,
  7195,
  7592,
  7768,
  7871,
  7965,
  8100,
  8684,
  8893,
  9084,
  8668,
  10009,
  9816,
  9609,
  8908,
  8723,
  8576,
  9047,
  9274,
  9309,
  9276,
  9627,
  9775,
  9786,
  10187,
  10443,
  10927,
  11259,
  11578,
  11708,
  12022,
  12117,
  12378,
  13388,
  13656,
  15372,
  15667,
  14221,
  14315,
  14040,
  13936,
  13687,
  13403,
  13394,
  12891,
  12999,
  13728,
  13564,
  13314,
  13210,
  13261,


In [226]:
# we use graph here
"""
 - graph here is the distance matrix
 - nb ants is number of ants
 - nb iterations is number of total iterations
 - evaporation rate speaks for itself
 - beta is a parameter by which we control the importance of the edges cost in probability evaluation
 - alpha is a parameter by which we control the importance of the pheromones trails left by ants in probability evaluation
"""
def aoc_tsp(
    graph: np.ndarray,
    nb_ants:int,
    nb_iterations:int,
    evaporation_rate:float = 0.5,
    alpha:float = 1.,
    beta:float = 1.,
    Q:float=1.
)-> Tuple[np.ndarray,int]:
    graph = np.array(graph)

    np.random.seed(seed = int(time.time()))
    
    nb_nodes = len(graph)
    pheromones = np.ones((nb_nodes,nb_nodes))
    
    optimal_length = np.inf
    optimal_path = []

    for iteration in range(nb_iterations):
        print(f"iteration number {iteration}")
        paths = []
        path_lengths = []
        for ant in range(nb_ants):
            
            current_city = np.random.randint(0, nb_nodes)
            visited = [False] * nb_nodes
            visited[current_city] = True
            path = [current_city]
            path_length = 0
            c=0
            while False in visited :
                
                unvisited = np.where(np.logical_not(visited))[0]
                probabilities = np.zeros(len(unvisited))

                for i, unvisited_city in enumerate(unvisited):
                    probabilities[i] = pheromones[current_city, unvisited_city]**alpha / (graph[current_city,unvisited_city])**beta

                probabilities /= np.sum(probabilities)
                
                probabilities /=np.sum(probabilities)
                
                next_city = np.random.choice(unvisited,p=probabilities)
                
                path.append(next_city)
                path_length+= graph[current_city,next_city]
                
                visited[next_city] = True
                current_city = next_city
            
            path_length+=graph[path[-1],path[0]]
            #   add the newly found path to our archive
            paths.append(path)
            path_lengths.append(path_length)
            
            #   update the optimal cost and path
            if path_length<optimal_length:
                optimal_length = path_length
                optimal_path = path
        
        # update the pheromones for the next tour
        pheromones*=evaporation_rate
        for path_,path_length_ in zip(paths,path_lengths):
            for i in range(nb_nodes-1) :
                pheromones[path_[i],path[i+1]] += Q/path_length_
            pheromones[path_[-1],path[0]] += Q/path_length_
    
    return optimal_path,optimal_length

In [227]:
# plotting
def plot_graph(
    n_points:int,
    best_path:List,
    points:np.ndarray
):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], c='r', marker='o')

    for i in range(n_points-1):
        ax.plot([points[best_path[i], 0], points[best_path[i+1], 0]],
                [points[best_path[i], 1], points[best_path[i+1], 1]],
                [points[best_path[i], 2], points[best_path[i+1], 2]],
                c='g', linestyle='-', linewidth=2, marker='o')

    ax.plot([points[best_path[0], 0], points[best_path[-1], 0]],
            [points[best_path[0], 1], points[best_path[-1], 1]],
            [points[best_path[0], 2], points[best_path[-1], 2]],
            c='g', linestyle='-', linewidth=2, marker='o')

    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()


In [229]:
optimal_path, optimal_length = aoc_tsp(
    graph=graph,
    nb_ants=200,
    nb_iterations=200,
)

optimal_length, optimal_path

# if GR_READ:
#     plot_graph(
#         n_points= len(graph),
#         best_path=optimal_length,
#         points=coords
#     )

iteration number 0
iteration number 1
iteration number 2
iteration number 3
iteration number 4
iteration number 5
iteration number 6
iteration number 7
iteration number 8
iteration number 9
iteration number 10
iteration number 11
iteration number 12
iteration number 13
iteration number 14
iteration number 15
iteration number 16
iteration number 17
iteration number 18
iteration number 19
iteration number 20
iteration number 21
iteration number 22
iteration number 23
iteration number 24
iteration number 25
iteration number 26
iteration number 27
iteration number 28
iteration number 29
iteration number 30
iteration number 31
iteration number 32
iteration number 33
iteration number 34
iteration number 35
iteration number 36
iteration number 37
iteration number 38
iteration number 39
iteration number 40
iteration number 41
iteration number 42
iteration number 43
iteration number 44
iteration number 45
iteration number 46
iteration number 47
iteration number 48
iteration number 49
iteration 

(298192,
 [78,
  43,
  69,
  93,
  123,
  122,
  106,
  107,
  103,
  115,
  113,
  111,
  109,
  108,
  119,
  112,
  110,
  97,
  98,
  130,
  129,
  133,
  64,
  54,
  53,
  60,
  57,
  40,
  61,
  66,
  67,
  48,
  31,
  72,
  49,
  84,
  91,
  39,
  68,
  89,
  86,
  88,
  65,
  17,
  25,
  15,
  127,
  41,
  32,
  19,
  47,
  45,
  12,
  21,
  58,
  34,
  38,
  8,
  28,
  29,
  36,
  70,
  87,
  136,
  59,
  46,
  11,
  14,
  9,
  35,
  62,
  73,
  90,
  50,
  71,
  76,
  124,
  126,
  92,
  104,
  114,
  80,
  131,
  128,
  121,
  94,
  55,
  18,
  5,
  27,
  42,
  16,
  56,
  4,
  6,
  20,
  13,
  44,
  7,
  10,
  1,
  0,
  26,
  22,
  2,
  3,
  105,
  117,
  118,
  116,
  101,
  132,
  74,
  85,
  83,
  82,
  135,
  134,
  102,
  100,
  99,
  125,
  96,
  75,
  95,
  79,
  120,
  77,
  81,
  63,
  51,
  52,
  33,
  24,
  37,
  30,
  23])